In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
!wget https://github.com/tensorflow/tensorflow/archive/v2.4.1.zip
!unzip v2.4.1.zip &> 0
!mv tensorflow-2.4.1/ tensorflow/
import sys
# We add this path so we can import the speech processing modules.
sys.path.append("/content/tensorflow/tensorflow/examples/speech_commands/")
import input_data
import models
import numpy as np
import glob
import os
import re
import shutil
from google.colab import files    # To upload the keyword recordings that we have selected
!pip install ffmpeg-python &> 0
!apt-get update && apt-get -qq install xxd

In [ ]:
# Get the 'speech_commands' dataset so we can build our dataset ontop of the 'speech_commands' dataset 
!wget https://storage.googleapis.com/download.tensorflow.org/data/speech_commands_v0.02.tar.gz

# Create a 'dataset' folder
DATASET_DIR =  'dataset/'
!mkdir dataset

# Extract the 'speech_commands' dataset and store it to the 'dataset' folder
!tar -xf speech_commands_v0.02.tar.gz -C 'dataset'
!rm -r -f speech_commands_v0.02.tar.gz

In [ ]:
# Upload your keyword recordings
uploaded = files.upload()

In [ ]:
# Convert the ogg files to wav files
!mkdir wavs
!find *.ogg -print0 | xargs -0 basename -s .ogg | xargs -I {} ffmpeg -i {}.ogg -ar 16000 wavs/{}.wav
!rm -r -f *.ogg

# Use pete warden's tool to only extract 1 second clips
!mkdir trimmed_wavs
!git clone https://github.com/petewarden/extract_loudest_section.git
!make -C extract_loudest_section/
!/tmp/extract_loudest_section/gen/bin/extract_loudest_section 'wavs/*.wav' trimmed_wavs/
!rm -r -f /wavs

In [ ]:
# Change the current directory to where the 1 second clip wav files are located
os.chdir('/content/trimmed_wavs/')

search_path_help = os.path.join('help_*.wav')   
search_path_send = os.path.join('send_*.wav')

# Count how many files for the keyword 'help' exist in the 'trimmed_wavs' folder
helpCount = 0
for file in glob.glob(search_path_help):
  helpCount+=1

# Count how many files for the keyword 'send' exist in the 'trimmed_wavs' folder
sendCount = 0
for file in glob.glob(search_path_send):
  sendCount += 1

print("Help files count --> ",helpCount)
print("Send files count --> ",sendCount)

# Change the directory back to the initial location
os.chdir('/content/')

In [ ]:
# Store the wav files to the dataset folder
data_index = {}
os.chdir('trimmed_wavs')
search_path = os.path.join('*.wav')
for wav_path in glob.glob(search_path):
    original_wav_path = wav_path
    parts = wav_path.split('_')
    if len(parts) > 2:
        wav_path = parts[0] + '_' + ''.join(parts[1:])
    matches = re.search('([^/_]+)_([^/_]+)\.wav', wav_path)
    if not matches:
        raise Exception('File name not in a recognized form:"%s"' % wav_path)
    word = matches.group(1).lower()
    instance = matches.group(2).lower()
    if not word in data_index:
      data_index[word] = {}
    if instance in data_index[word]:
        raise Exception('Audio instance already seen:"%s"' % wav_path)
    data_index[word][instance] = original_wav_path

output_dir = os.path.join('..', 'dataset')
try:
    os.mkdir(output_dir)
except:
    pass
for word in data_index:
  word_dir = os.path.join(output_dir, word)
  try:
      os.mkdir(word_dir)
      print('Created dir: ' + word_dir)
  except:
      print('Storing in existing dir: ' + word_dir)
  for instance in data_index[word]:
    wav_path = data_index[word][instance]
    output_path = os.path.join(word_dir, instance + '.wav')
    shutil.copyfile(wav_path, output_path)
os.chdir('..')
!rm -r -f trimmed_wavs